In [ ]:
##si prende l'evaporazione dal modello di PM e invece la traspirazione si prende da Prospero, si sommano le due

# **GEOET - EvapoTranspiration Penman-MonteithFAO & Prospero**

- Author Concetta D'Amato, Niccolò Tubini, Michele Bottazzi and Riccardo Rigon 

This work is licensed under a Creative Commons Attribution-NonCommercial 4.0 International License
____

### *What about Prospero*

Bottazzi, Michele. n.d. “[Transpiration Theory and the Prospero Component of GEOframe](https://paperpile.com/shared/yN2wjv).” Supervised by R. Rigon and G. Bertoldi. Ph.D., Civil, Environmental and Mechanical Engineering of University of Trento. 

More informations http://abouthydrology.blogspot.com/2020/07/michele-bottazzi-phd-thesis.html

In [ ]:
import os
import warnings
import datetime
import warnings
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import rc
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.dates as mdates
from IPython.display import Image
from GEOSPACE_Output import*
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
oms_project_path = os.path.dirname(os.getcwd())

In [ ]:
Image(oms_project_path+"/Images/Pro.png")

## **Evaporation from soil**

Evaporation from soil is computed with a **Penman-Monteith FAO model**, with specific coefficients for soil evaporation depending on the case study and considering the *net radiation* that overcome plant and affects the soil.

\begin{equation}
ET_{0} =  \dfrac{0.408\cdot \Delta\cdot (R_{n} - G)+\gamma\cdot u_{2}\cdot(e_{s}-e)\cdot\frac{C_{p}}{T+273}}{\lambda(\Delta+\gamma\cdot(1+C_{d}\cdot u_{2}))}
\end{equation}

where:
- $ET_{0}$ is expressed in [$mm \cdot day^{−1}$] or [$mm\cdot hour^{−1}$]; 
- $R_{n}$ is the net radiation expressed in [$MJ \cdot m^{2} \cdot day^{−1}$] or [$MJ \cdot m^{2}\cdot h^{−1}$]; 
- $G$ is the soil heat flux at the soil surface in [$MJ \cdot m^{2} \cdot day^{−1}$] or [$MJ \cdot m^{2}\cdot h^{−1}$]; 
- $T$ is the mean daily or hourly air temperature expressed in (°C) ; 
- $u_{2}$ is the wind speed in ($m\cdot s^{−1}$); 
- $e_{s}$ is the mean saturation vapor-pressure expressed in ($kP_{a}$); 
- $e$ is the mean actual vapor-pressure; 
- $\Delta$ is the slope of the saturation vapor-pressure curve expressed in ($kP_{a}\dot °C^{−1}$); 
- $\gamma$ is the psychometric constant expressed in ($kP_{a} \cdot$ °$C^{−1}$); 
- $\lambda$ is the latent heat constant constant expressed in ($J \cdot$ $kg^{−1}$); 
- $C_{d}$ is a coefficient equal to 0.34 and $C_{p}$ is a coefficient equal to 900 in the case of a daily time step and equal to 37 in the case of a hourly time step.

## **Transpiration**

In Prospero the transpiration is computed starting from the [Schymanski and Or method](https://hess.copernicus.org/articles/21/685/2017/)  and modified in order to include the dependence from transpiring surface, the conductance model of the stomata, other than the mass conservation.  

Prospero solves a four linear equation system and compute the leaf temperature, the vapour pressure, the sensible heat and the latent heat:

\begin{equation}
   \begin{cases}
   T_{\Delta}=\frac{\gamma}{a_{sH}\ C_{Dr}\gamma+a_{sE}\ C_{Da}\Delta_{e{T}}}\cdot\frac{R_{s}-R_{ll}-S_{nk}}{\rho_{a} \bar u c_{p}}-\frac{C_{Da}}{a_{sH}\ C_{Dr}\gamma+a_{sE}\ C_{Da}\Delta_{e{T}}}\cdot \delta e(z)\\e_{\Delta}=\frac{\gamma\Delta_{e{T}}}{a_{sH}\ C_{Dr}\gamma+a_{sH}\ C_{Da}\Delta_{e{T}}}\cdot\frac{R_{s}-R_{ll}-S_{nk}}{\rho_{a} \bar u c_{p}}-\frac{a_{sH}\ C_{Dr}\gamma+(a_{sE}-1)\ C_{Da}\Delta_{e{T}}}{a_{sH}\ C_{Dr}\gamma+a_{sH}\ C_{Da}\Delta_{e{T}}}\cdot \delta e(z)\\H=\frac{C_{Dr}\gamma}{a_{sH} C_{Dr}\gamma+a_{sE}C_{Da}\Delta_{e{T}}}\cdot(R_{s}-R_{ll}-S_{nk})+\frac{\rho_{a}\bar u\ c_{p}\ C_{Dr}\ C_{Da}}{a_{sH}\ C_{Dr}\gamma+a_{sE}\ C_{Da}\Delta_{e{T}}}\cdot \delta e(z)\\\lambda E_{T}=\frac{C_{Da}\Delta_{e{T}}}{ C_{Dr}\gamma+a_{sE}C_{Da}\Delta_{e{T}}}\cdot(R_{s}-R_{ll}-S_{nk})+\frac{\rho_{a}\bar u\ c_{p}\ a_{sE}\ C_{Dr}\ C_{Da}\ +(a_{sE}-1)\ C_{Da}^2\Delta_{e{T}}\gamma^{-1} }{a_{sH}\ C_{Dr}\gamma+a_{sE}\ C_{Da}\Delta_{e{T}}}\cdot \delta e(z)
   \end{cases} 
\end{equation}

Prospero computes the actual evapotranspiration multiplying the Potential Evapotranspiration for a total stress factor $g_{s}$.
\begin{equation}
ET_{eff}=ET_{pot}\cdot g_{s}
\end{equation}

\begin{equation}
g_{s} = g_{s}max\cdot f(R_{par})\cdot f(T_{a})\cdot f(VPD) \cdot K_{s}
\end{equation}

where:
- $g_{s}max $ is the conductance without any kind of stress and in well-watered conditions;
- $f(R_{par})$ is the stress coefficient as a function of photosynthetically active radiation;
- $f(T_{a})$ is the stress coefficient as a function of the air temperature;
- $f(VPD) $ is the stress coefficient as a function of the vapor pressure deficit;
- $K_{s} $ is the stress coefficient as a function of the water content.

## Input required:
- Dem
- Centroids
- Air temperature
- Wind velocity
- Relative humidity
- Shortwave direct radiation
- Shortwave diffuse radiation
- Longwave radiation
- Net Radiation
- Atmospheric pressure
- Leaf Area Index (LAI)
- Soil water content
- Ground Heat Flux

## Prospero model Output

- Latent heat both for sunlight and shaded canopy [$W/m^{2}$];
- Evaporation, Traspiration, Evapotranspiration flux [$W/m^{2}$];
- Evapotranspiration [$mm$];
- Leaves temperature both for sunlight and shaded canopy [$W/m^{2}$];
- Sensible heat both for sunlight and shaded canopy [$W/m^{2}$];
- Fraction of highlighted canopy [-];
- Shortwave radiation absorbed both for sunlight and shaded canopy [$W/m^{2}$];

In [ ]:
os.chdir(oms_project_path+'/output/Cavone/1')
#os.listdir()

### **Transpiration  $[W m^{−2}]$**

In [ ]:
df2 = pd.read_csv('FluxTranspiration_1_waterstress.csv',skiprows=6, sep=',', parse_dates=[0], na_values=-9999,usecols=[1,2])
df2.columns = ['Datetime','FluxTranspiration']
fig = px.line(df2, x='Datetime', y='FluxTranspiration')
fig.update_traces(line_color='green')
fig.show()

In [ ]:

show_E_T('FluxEvapoTranspiration_1_waterstress.csv',
         'FluxEvaporation_1_waterstress.csv',
         'FluxTranspiration_1_waterstress.csv')

In [ ]:
##BISOGNEREBBE CREARE UN LAI SINUSOIDALE CHE RICREA L'ANDAMENTO STAGIONALE DELLA PRESENZA DI FOGLIA CHE DIPENDE IN BASE AL PERIODO DELL'ANNO PER AVERE DEI DATI PIù CORRETTI
#LA PROF HA CREATO UN L

In [ ]:
#BISOGNA CREARE I FIL E FARLO PARTIRE
show_stress('FluxTranspiration_1_potential.csv',
            'FluxTranspiration_1_environmentalstress.csv',
            'FluxTranspiration_1_totalstress.csv',
            'FluxTranspiration_1_waterstress.csv')

In [ ]:
df = pd.read_csv('Evaporation_FAO_1.csv',skiprows=6, sep=',', parse_dates=[0], na_values=-9999,usecols=[1,2])
df.columns = ['Datetime','Evaporation']
fig = px.line(df, x='Datetime', y='Evaporation', title='Evaporation PenmanMonteithFAO')
fig.update_traces(line_color='green')
#fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
df3 = pd.read_csv('StressedEvaporation_FAO_1.csv',skiprows=6, sep=',', parse_dates=[0], na_values=-9999,usecols=[1,2])
df3.columns = ['Datetime','Evaporation']
fig = px.line(df3, x='Datetime', y='Evaporation', title='Actual Evaporation PenmanMonteithFAO')
fig.update_traces(line_color='green')
#fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.line()
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['Evaporation'], mode='lines', name='Potential E'))
fig.add_trace(go.Scatter(x=df3['Datetime'], y=df3['Evaporation'], mode='lines', name='Actual E'))

fig.update_layout(
    title='Compare Potential and Actual PenmanMonteithFAO Evaporation ',
    xaxis_title="DateTime",
    font_family="Times New Roman",
    font_color="Black",
    title_font_family="Times New Roman",
    title_font_color="Black",
    yaxis_title="EvapoTranspiration [mm]",
    #legend_title="Date",
    font=dict(size=12))
fig.show()

In [ ]:
show_stress('Evaporation_1_potential.csv',
            'Evaporation_1_environmentalstress.csv',
            'Evaporation_1_totalstress.csv',
            'Evaporation_1_waterstress.csv')

In [ ]:
df = pd.read_csv('LT_Sun_1_waterstress.csv',skiprows=6, sep=',', parse_dates=[0], na_values=-9999,usecols=[1,2])
df.columns = ['Datetime','LeafTemperature']
fig = px.line(df, x='Datetime', y='LeafTemperature', title='Leaf Temperature Sun [K]')
fig.update_traces(line_color='green')
fig.show()

In [ ]:
df = pd.read_csv('LT_Shade_1_waterstress.csv',skiprows=6, sep=',', parse_dates=[0], na_values=-9999,usecols=[1,2])
df.columns = ['Datetime','LeafTemperature']
fig = px.line(df, x='Datetime', y='LeafTemperature', title='Leaf Temperature Shadow [K]')
fig.update_traces(line_color='green')
fig.show()